In [ ]:
import pandas as pd
speed_data_path = "speedtime.xlsx"
speed_df = pd.read_excel(speed_data_path, sheet_name=0)
speed_df.dropna(axis=1, how='all', inplace=True)
speed_df = speed_df.select_dtypes(include=["number"])
summary_data = {}
for col in speed_df.columns:
    valid_data = speed_df[col].dropna()

    if not valid_data.empty:
        avg_speed = valid_data.mean()
        percentiles = valid_data.quantile([0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).to_dict()

        summary_data[col] = {
            "Average Speed": avg_speed,
            "25th Percentile": percentiles.get(0.25, None),
            "50th Percentile (Median)": percentiles.get(0.50, None),
            "75th Percentile": percentiles.get(0.75, None),
            "90th Percentile": percentiles.get(0.90, None),
            "95th Percentile": percentiles.get(0.95, None),
            "99th Percentile": percentiles.get(0.99, None)
        }


summary_df = pd.DataFrame.from_dict(summary_data, orient="index")
print(summary_df)
summary_df.to_excel("speed_summary.xlsx", index=True)


         Average Speed  25th Percentile  50th Percentile (Median)  \
TIME        249.500000       125.250000                249.500000   
1             1.303000         0.701312                  1.579233   
2             0.836021         0.241509                  0.709373   
3             1.439759         1.242087                  1.740878   
4             1.353851         1.058546                  1.551093   
5             1.193627         0.542484                  1.403262   
6             1.020853         0.150278                  1.139283   
7             1.624493         1.383413                  1.755556   
8             1.277860         1.026088                  1.510382   
9             1.153400         0.916538                  1.289334   
10            1.267007         0.890859                  1.423040   
11            1.624493         1.383413                  1.755556   
12            0.836021         0.241509                  0.709373   
13            1.040784         0.6

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
def calculate_power_energy_battery(speed_file,battery_voltage,avg_trip_time_sec,avg_trip_time_min):
    speed_df = pd.read_csv(speed_file)
    C_d = 0.24#drag coeff
    rho = 1.2#air density(kg/m³)
    A = 0.0682#frontal area (m²)
    C_r = 0.01#rolling resistance coeff
    M = 93#mass of rider+cycle(kg)
    g = 9.8#gravitational acceleration(m/s²)
    factor_inertia = 1.05#factor for inertia effects
    theta = np.radians(1)#assume a small gradient (~1 degree)

    #extract speed columns
    speed_columns=speed_df.columns[1:-1]
    X=speed_df[speed_columns].dropna().values
    #apply KMeans clustering to determine cluster-based speeds
    kmeans=KMeans(n_clusters=3,random_state=42,n_init=10)
    kmeans.fit(X)
    cluster_centers=kmeans.cluster_centers_
    speed_df["Cluster Avg Speed"]=[cluster_centers[label].mean() for label in kmeans.labels_]

    #compute power using the cluster avg Speed
    speed_df["Cluster Power (W)"]=((0.5*C_d*rho*A*(speed_df["Cluster Avg Speed"]**2)) +
                                     (C_r*M*g) +
                                     (M*g*np.sin(theta)) +
                                     (factor_inertia*M*speed_df["Cluster Avg Speed"]))*speed_df["Cluster Avg Speed"]

    #cmpute average power across speed data
    speed_df["Avg Power (W)"]=((0.5*C_d*rho*A*(speed_df.iloc[:,1]**2))+
                                  (C_r*M*g)+
                                  (M*g*np.sin(theta))+
                                  (factor_inertia*M*speed_df.iloc[:,1]))*speed_df.iloc[:,1]

    #calculate total average power using the cluster-based speeds
    total_avg_cluster_power=speed_df["Cluster Power (W)"].mean()
    total_avg_power=speed_df["Avg Power (W)"].mean()

    #convert trip time to hours
    avg_trip_time_hours_sec=avg_trip_time_sec/3600#convert seconds to hours
    avg_trip_time_hours_min=avg_trip_time_min/60#convert minutes to hours

    #calculate energy required(Wh)
    energy_required_sec_wh=total_avg_cluster_power*avg_trip_time_hours_sec
    energy_required_min_wh=total_avg_cluster_power*avg_trip_time_hours_min

    #calculate battery capacity(Ah)
    battery_capacity_ah_sec = energy_required_sec_wh / battery_voltage
    battery_capacity_ah_min = energy_required_min_wh / battery_voltage

    return {
        "Total Avg Cluster Power(W)":total_avg_cluster_power,
        "Total Avg Power(W)":total_avg_power,
        "Energy Required(Wh)for 250 sec":energy_required_sec_wh,
        "Energy Required(Wh)for 10 min":energy_required_min_wh,
        "Battery Capacity(Ah)for 250 sec":battery_capacity_ah_sec,
        "Battery Capacity(Ah)for 10 min":battery_capacity_ah_min,
        "Updated DataFrame":speed_df
    }
speed_analysis_path = "Speed_Analysis.csv"
battery_voltage = 36
avg_trip_time_sec = 250
avg_trip_time_min = 10
results=calculate_power_energy_battery(speed_analysis_path,battery_voltage,avg_trip_time_sec,avg_trip_time_min)
results["Updated DataFrame"].to_csv("Final_Power_Energy_Analysis.csv",index=False)


for key, value in results.items():
    if key != "Updated DataFrame":
        print(f"{key}: {value}")
print("Results saved as 'Final_Power_Energy_Analysis.csv'")

Total Avg Cluster Power (W): 255.28836926994148
Total Avg Power (W): 174.01007119537377
Energy Required (Wh) for 250 sec: 17.72835897707927
Energy Required (Wh) for 10 min: 42.54806154499025
Battery Capacity (Ah) for 250 sec: 0.49245441602997975
Battery Capacity (Ah) for 10 min: 1.1818905984719512
Results saved as 'Final_Power_Energy_Analysis.csv'
